# Amazon Bedrock AgentCore Policy - Lambda Target Setup

## Overview

This notebook sets up the infrastructure for testing **Amazon Bedrock AgentCore Policy** with a **Lambda Target**.

### What This Notebook Creates

| Resource | Description |
|----------|-------------|
| Lambda Function | `RefundLambda` - Sample refund processing function |
| AgentCore Gateway | MCP protocol endpoint with JWT Authorizer |
| Gateway Target | Lambda tool connected to Gateway |
| Cognito User Pool | Authentication container |
| Cognito App Client | M2M client for token requests |
| Cognito Resource Server | OAuth2 scope definition |

### Architecture

```
┌─────────────────┐             ┌───────────────────────┐             ┌─────────────┐
│   Amazon        │  JWT Token  │  AgentCore Gateway    │             │   Lambda    │
│   Cognito       │────────────>│  + JWT Authorizer     │────────────>│   Target    │
│                 │             │                       │             │   (Tool)    │
└─────────────────┘             └───────────────────────┘             └─────────────┘
```

> **Next Step**: After completing this notebook, run `02-Policy-Enforcement.ipynb` to test Cedar policies.

---

## Part 1: Environment Setup

### Setup Virtual Environment (Recommended)

```bash
cd ../00_setup
chmod +x create_uv_virtual_env.sh
./create_uv_virtual_env.sh AgentCorePolicy
```

Then select the `AgentCorePolicy` kernel in Jupyter.

In [ ]:
import json
import sys
import time
from pathlib import Path

import boto3

# Add parent directory to path for common imports
sys.path.insert(0, str(Path.cwd().parent))

# Import utility functions from common folder
from common.auth_utils import (
    get_bearer_token,
    decode_token,
    make_gateway_request,
    analyze_response,
)
from common.gateway_utils import (
    validate_and_fix_gateway_authorizer,
)

print("✓ Libraries loaded")

---

## Part 2: Create Gateway and Cognito

Run `setup-gateway.py` to create all required AWS resources.

> ⚠️ Skip this cell if `gateway_config.json` already exists.

In [ ]:
# Create Gateway and Cognito (only if gateway_config.json doesn't exist)
config_file = Path.cwd() / "gateway_config.json"

if not config_file.exists():
    print("gateway_config.json not found. Creating Gateway and Cognito...")
    %run setup-gateway.py --region us-east-1
else:
    print(f"✓ gateway_config.json already exists: {config_file}")

### Created Resources

#### AgentCore Gateway
![Gateway.png](img/Gateway.png)

#### Lambda Function
![refund_lambda.png](img/refund_lambda.png)

#### Cognito User Pool
![cognito_user_pool](img/cognito_user_pool.png)

#### Cognito App Client
![Cognito_App_Clients.png](img/Cognito_App_Clients.png)

#### Cognito Domain
![Cognito_Domain.png](img/Cognito_Domain.png)

#### Cognito Resource Server
![Cognito_Resource_Server.png](img/Cognito_Resource_Server.png)

---

## Part 3: Load Configuration

In [ ]:
# Load configuration file
config_path = Path.cwd() / "gateway_config.json"

with open(config_path, "r") as f:
    CONFIG = json.load(f)

# Extract key values
REGION = CONFIG["region"]
GATEWAY_URL = CONFIG["gateway_url"]
GATEWAY_ID = CONFIG["gateway_id"]
GATEWAY_ARN = CONFIG["gateway_arn"]
USER_POOL_ID = CONFIG["client_info"]["user_pool_id"]
CLIENT_ID = CONFIG["client_info"]["client_id"]
CLIENT_SECRET = CONFIG["client_info"]["client_secret"]
TOKEN_ENDPOINT = CONFIG["client_info"]["token_endpoint"]

print("✓ Configuration loaded")
print(f"  REGION: {REGION}")
print(f"  GATEWAY_ID: {GATEWAY_ID}")
print(f"  GATEWAY_URL: {GATEWAY_URL}")
print(f"  GATEWAY_ARN: {GATEWAY_ARN}")
print(f"  USER_POOL_ID: {USER_POOL_ID}")
print(f"  CLIENT_ID: {CLIENT_ID}")
print(f"  TOKEN_ENDPOINT: {TOKEN_ENDPOINT}")

---

## Part 4: Initialize AWS Clients

In [ ]:
session = boto3.Session(region_name=REGION)

sts_client = session.client("sts")
gateway_control_client = session.client("bedrock-agentcore-control", region_name=REGION)

ACCOUNT_ID = sts_client.get_caller_identity()["Account"]

print("✓ AWS clients initialized")
print(f"  Account ID: {ACCOUNT_ID}")
print(f"  Region: {REGION}")

---

## Part 5: Validate Gateway Authorizer

Ensure the Gateway JWT Authorizer is correctly configured for Cognito.

In [ ]:
# Validate and fix Gateway Authorizer if needed
scope = CONFIG.get("client_info", {}).get("scope", "")
print(f"Scope: {scope}")

validate_and_fix_gateway_authorizer(
    gateway_control_client=gateway_control_client,
    gateway_id=GATEWAY_ID,
    region=REGION,
    user_pool_id=USER_POOL_ID,
    client_id=CLIENT_ID,
    scope=scope
)

---

## Part 6: Verify Setup

Test that the Gateway and Lambda target work correctly (without policy enforcement).

In [ ]:
# Get Bearer Token
print("Getting Bearer Token...")
token = get_bearer_token(
    token_endpoint=TOKEN_ENDPOINT,
    client_id=CLIENT_ID,
    client_secret=CLIENT_SECRET
)

print(f"✓ Token received")
print(f"  Token (first 50 chars): {token[:50]}...")

# Decode and show claims
claims = decode_token(token)
print(f"\n🔍 Token Claims:")
print(f"  client_id: {claims.get('client_id', 'N/A')}")
print(f"  scope: {claims.get('scope', 'N/A')}")

In [ ]:
# Test Gateway request (without policy - should work or fail based on current policy state)
print("\nTesting Gateway request...")
print("=" * 70)

result = make_gateway_request(
    gateway_url=GATEWAY_URL,
    bearer_token=token,
    tool_name="RefundToolTarget___refund",
    arguments={"amount": 100, "orderId": "test-setup-verification"}
)

print("\nResponse:")
print(json.dumps(result, indent=2, ensure_ascii=False))

outcome = analyze_response(result)
print(f"\nResult: {outcome}")

if outcome == "ALLOWED":
    print("\n✓ Setup verification successful! Gateway and Lambda are working.")
elif outcome == "DENIED":
    print("\n⚠️  Request was denied by policy. This is expected if policies are attached.")
    print("   Run 02-Policy-Enforcement.ipynb to configure and test policies.")
else:
    print(f"\n⚠️  Unexpected result: {outcome}")
    print("   Check the response above for details.")

---

## Summary

This notebook has set up:

✅ Lambda Function (`RefundLambda`)  
✅ AgentCore Gateway with JWT Authorizer  
✅ Gateway Target (Lambda tool)  
✅ Cognito User Pool, App Client, and Resource Server  
✅ Validated Gateway Authorizer configuration  

### Next Step

Run **`02-Policy-Enforcement.ipynb`** to:
- Create Policy Engine and attach to Gateway
- Test Cedar policies for fine-grained access control
- Learn department-based and group-based access patterns